In [2]:
import pandas as pd
import sys
import os
from pathlib import Path
from config import DBConfig
import sqlite3

project_root = os.path.abspath('../../')
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.data.db import DBConnection, execute_query, get_pitcher_data

In [3]:
class NotebookDBConnection(DBConnection):
    def __init__(self):
        db_path = os.path.join(project_root, 'data', 'pitcher_stats.db')
        super().__init__(db_name=db_path)

In [4]:
pd.set_option('display.max_columns', 150)

In [5]:
data_path = os.path.join('..', '..', 'data', 'output', 'pitcher_prediction_dataset.csv')

df = pd.read_csv(data_path)

In [6]:
df.head()

,game_db_id,pitcher_id,player_name,statcast_id,game_id,game_date,season,opponent_team_id,strikeouts,hits,walks,home_runs,release_speed_mean,release_speed_max,release_spin_rate_mean,swinging_strike_pct,called_strike_pct,zone_rate,last_3_games_strikeouts_avg,last_5_games_strikeouts_avg,last_10_games_strikeouts_avg,last_3_games_velo_avg,last_5_games_velo_avg,last_10_games_velo_avg,last_3_games_swinging_strike_pct_avg,last_5_games_swinging_strike_pct_avg,last_10_games_swinging_strike_pct_avg,days_rest,team_changed,last_3_games_strikeouts_std,last_5_games_strikeouts_std,last_3_games_velo_std,last_5_games_velo_std,last_3_games_swinging_strike_pct_std,last_5_games_swinging_strike_pct_std,last_3_games_called_strike_pct_avg,last_5_games_called_strike_pct_avg,last_3_games_zone_rate_avg,last_5_games_zone_rate_avg,trend_3_strikeouts,trend_5_strikeouts,trend_3_release_speed_mean,trend_5_release_speed_mean,trend_3_swinging_strike_pct,trend_5_swinging_strike_pct,momentum_3_strikeouts,momentum_5_strikeouts,momentum_3_release_speed_mean,momentum_5_release_speed_mean,momentum_3_swinging_strike_pct,momentum_5_swinging_strike_pct,pitch_entropy,prev_game_pitch_entropy,career_so_avg,career_so_std,career_so_per_batter,career_so_consistency,prev_so_deviation,so_deviation_3g_avg,so_deviation_5g_avg,is_home_game,home_away_so_exp,opponent_whiff_rate,opponent_chase_rate,opponent_zone_contact_rate,opponent_k_vs_avg,opponent_whiff_vs_avg,matchup_advantage,recency_weighted_matchup,so_vs_NYY,so_vs_SF,so_vs_WSH,so_vs_CLE,so_vs_AZ,so_vs_TEX,so_vs_STL,so_vs_HOU,so_vs_KC,so_vs_TOR,so_vs_DET,so_vs_TB,so_vs_BAL,so_vs_MIL,so_vs_LAA,so_vs_MIA,so_vs_LAD,so_vs_SEA,so_vs_PHI,so_vs_COL,so_vs_CWS,so_vs_NYM,so_vs_CIN,so_vs_SD,so_vs_OAK,so_vs_PIT,so_vs_CHC,so_vs_BOS,so_vs_ATL,so_vs_MIN
0,84454,1268,"Urquidy, José",664353,662131,2022-05-28,2022,NaN,4,12,2,1,87.576344,95.0,2197.333333,9.677419,9.677419,1.0,4.666667,4.6,3.6,0.0,0.0,0.0,11.769051,11.711003,9.424602,6,0,4.725816,3.507136,0.0,0.0,7.508279,5.993520,16.556724,15.680066,1.0,1.0,0.666667,2.0,0.454834,18.814914,0.531391,4.572803,5.833333,5.000000,89.806828,89.653263,14.034048,12.383918,0.0,0.0,4.216216,2.800150,0.0,0.463184,2.233658,0.202476,0.166856,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.164384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,84458,1268,"Urquidy, José",664353,662692,2022-06-03,2022,NaN,6,5,1,1,89.481319,95.2,2182.901099,12.087912,9.890110,1.0,5.000000,4.2,3.4,0.0,0.0,0.0,12.722130,10.535376,8.853882,6,0,4.582576,3.420526,0.0,0.0,6.704000,5.615458,13.721924,14.726661,1.0,1.0,1.000000,1.6,-0.597026,18.099585,2.700115,3.362988,5.500000,4.800000,88.639217,89.001408,12.988232,11.706056,0.0,0.0,4.210526,2.762274,0.0,0.469867,-0.077216,0.323089,0.016683,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.164384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,84462,1268,"Urquidy, José",664353,662760,2022-06-08,2022,NaN,3,7,4,2,89.152747,94.9,2234.494505,7.692308,12.087912,1.0,6.666667,4.8,4.0,0.0,0.0,0.0,14.057832,11.414497,10.062673,5,0,3.055050,3.420526,0.0,0.0,5.630067,5.398994,12.304823,13.847540,1.0,1.0,4.333333,1.6,-0.440121,0.570354,6.527399,2.703647,6.000000,5.400000,88.926141,89.103529,12.671123,12.223568,0.0,0.0,4.256410,2.740706,0.0,0.478597,0.647826,0.934756,0.235608,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.164384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,84453,1268,"Urquidy, José",664353,661880,2022-06-14,2022,NaN,4,5,0,1,84.540476,94.3,2413.535714,15.476190,16.666667,1.0,4.333333,4.8,4.1,0.0,0.0,0.0,9.819213,11.589322,10.276349,6,0,1.527525,3.420526,0.0,0.0,2.201230,5.224278,10.551814,12.628759,1.0,1.0,-0.333333,1.4,-1.174763,-0.107329,-1.949838,2.625947,4.166667,4.800000,88.999537,89.034954,9.488361,10.982839,0.0,0.0,4.225000,2.712625,0.0,0.480837,-0.458426,0.037395,0.231733,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [7]:
df['so_vs_SF'].unique()

array([0.        , 1.        , 1.5       , 0.5       , 1.33333333,
       1.25      , 1.2       , 1.16666667, 4.        , 5.        ,
       3.66666667, 2.75      , 2.4       , 2.        , 6.        ,
       6.5       , 7.66666667, 3.        , 7.        , 0.66666667])

In [8]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print([table[0] for table in tables])

['teams', 'game_context', 'pitchers', 'game_stats', 'sqlite_sequence', 'pitch_mix', 'prediction_features', 'pitch_mix_features', 'team_stats']


In [9]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM teams")
    teams = cursor.fetchall()
    print(teams[0] )

('NYY', 'NYY', None)


In [10]:
from pybaseball import statcast_pitcher
from pybaseball import playerid_lookup
import pybaseball as pb

In [20]:
print(playerid_lookup('yamamoto', 'yoshinobu'))

  name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0  yamamoto  yoshinobu     808967  yamay001  yamamyo01          33825   

   mlb_played_first  mlb_played_last  
0            2024.0           2025.0  


In [12]:
data = statcast_pitcher('2025-03-27', '2025-04-01', player_id = 808967)

In [13]:
df = pd.DataFrame(data)

In [14]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,FS,2025-03-28,89.6,-1.76,5.30,"Yamamoto, Yoshinobu",679529,808967,strikeout,called_strike,NaN,NaN,NaN,NaN,13,Spencer Torkelson called out on strikes.,R,R,R,LAD,DET,S,2.0,None,2,2,2025,-0.53,0.44,-0.85,2.37,NaN,NaN,650402.0,2,5,Top,NaN,NaN,NaN,NaN,NaN,NaN,3.392809,-130.525850,-2.251716,-6.716830,25.116574,-26.844492,3.3,1.63,NaN,NaN,NaN,90.4,1387,6.5,778544,605131,518692,669242,571970,605141,624424,681624,606192,54.00,NaN,0.0,0.0,1.0,0.0,0.0,NaN,34,5,Split-Finger,0,2,2,0,2,0,2,0,Standard,Strategic,234,0.018,-0.234,NaN,NaN,NaN,0.234,NaN,-2,2,0.259,0.741,26,25,27,26,3,2,10,1.0,NaN,1.0,2.37,0.53,0.53,42.2
1,FF,2025-03-28,94.5,-1.76,5.29,"Yamamoto, Yoshinobu",679529,808967,None,called_strike,NaN,NaN,NaN,NaN,3,Spencer Torkelson called out on strikes.,R,R,R,LAD,DET,S,NaN,None,2,1,2025,-0.55,1.35,0.58,2.92,NaN,NaN,650402.0,2,5,Top,NaN,NaN,NaN,NaN,NaN,NaN,7.347047,-137.435010,-3.487813,-8.631429,30.781816,-14.531219,3.3,1.60,NaN,NaN,NaN,95.0,2193,6.6,778544,605131,518692,669242,571970,605141,624424,681624,606192,53.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,4,4-Seam Fastball,0,2,2,0,2,0,2,0,Standard,Strategic,216,0.000,-0.072,NaN,NaN,NaN,0.072,NaN,-2,2,0.259,0.741,26,25,27,26,3,2,10,1.0,NaN,1.0,1.21,0.55,0.55,41.1
2,CU,2025-03-28,76.7,-1.60,5.46,"Yamamoto, Yoshinobu",679529,808967,None,called_strike,NaN,NaN,NaN,NaN,1,Spencer Torkelson called out on strikes.,R,R,R,LAD,DET,S,NaN,None,2,0,2025,1.01,-1.28,-0.54,3.00,NaN,NaN,650402.0,2,5,Top,NaN,NaN,NaN,NaN,NaN,NaN,0.428422,-111.531939,4.221071,8.258677,22.289221,-43.822473,3.3,1.64,NaN,NaN,NaN,76.7,2712,6.5,778544,605131,518692,669242,571970,605141,624424,681624,606192,53.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,3,Curveball,0,2,2,0,2,0,2,0,Standard,Strategic,36,0.000,-0.048,NaN,NaN,NaN,0.048,NaN,-2,2,0.259,0.741,26,25,27,26,3,2,10,1.0,NaN,1.0,5.19,-1.01,-1.01,51.6
3,FC,2025-03-28,90.5,-1.83,5.25,"Yamamoto, Yoshinobu",679529,808967,None,ball,NaN,NaN,NaN,NaN,14,Spencer Torkelson called out on strikes.,R,R,R,LAD,DET,B,NaN,None,1,0,2025,0.50,0.71,0.84,1.79,NaN,NaN,650402.0,2,5,Top,NaN,NaN,NaN,NaN,NaN,NaN,5.634651,-131.742206,-4.265003,4.603606,27.452985,-23.193359,3.3,1.60,NaN,NaN,NaN,91.0,2456,6.5,778544,605131,518692,669242,571970,605141,624424,681624,606192,54.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,2,Cutter,0,2,2,0,2,0,2,0,Standard,Strategic,187,0.000,0.050,NaN,NaN,NaN,-0.050,NaN,-2,2,0.259,0.741,26,25,27,26,3,2,10,1.0,NaN,1.0,2.07,-0.50,-0.50,38.0
4,CU,2025-03-28,76.4,-1.48,5.56,"Yamamoto, Yoshinobu",679529,808967,None,ball,NaN,NaN,NaN,NaN,1,Spencer Torkelson called out on strikes.,R,R,R,LAD,

In [15]:
df.to_csv('statcast_pitcher_yamamoto.csv')

In [16]:
pitching_stats = pb.pitching_stats(2024, 2025, qual=0)

starters = pitching_stats[pitching_stats['GS'] >= 5].copy()

In [17]:
print(starters['IDfg'].nunique())

241


In [18]:
starters.head()

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA,botERA,botOvr CH,botStf CH,botCmd CH,botOvr CU,botStf CU,botCmd CU,botOvr FA,botStf FA,botCmd FA,botOvr SI,botStf SI,botCmd SI,botOvr SL,botStf SL,botCmd SL,botOvr KC,botStf KC,botCmd KC,botOvr FC,botStf FC,botCmd FC,botOvr FS,botStf FS,botCmd FS,botOvr,botStf,botCmd,botxRV100,Stf+ CH,Loc+ CH,Pit+ CH,Stf+ CU,Loc+ CU,Pit+ CU,Stf+ FA,Loc+ FA,Pit+ FA,Stf+ SI,Loc+ SI,Pit+ SI,Stf+ SL,Loc+ SL,Pit+ SL,Stf+ KC,Loc+ KC,Pit+ KC,Stf+ FC,Loc+ FC,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
319,10603,2024,Chris Sale,ATL,35,18,3,6.4,2.38,29,29,0,0,0,0,177.2,702,141,48,47,9,39,0,8,4,1,225,191,143,92,8,925,1893,2818,126,5,4,0,11.40,1.98,5.77,7.14,0.46,0.215,1.01,0.314,0.798,2.09,1.34,0.216,0.448,0.336,0.056,0.063,0.026,0.000,56.2,177.2,NaN,NaN,56.2,$51.3,2.46,2.64,4.04,-9.08,13.12,41.11,4.24,0.95,0.90,0.87,0.83,29,4.37,...,90.0,101.0,105.0,111.0,79.0,86.5,9.6,24.0,0.056,114.4,132.0,0.307,430,0.182,0.323,NaN,3.34,56.0,48.0,61.0,NaN,NaN,NaN,58.0,48.0,62.0,57.0,50.0,58.0,59.0,46.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,48.0,64.0,-0.49,95.0,109.0,102.0,NaN,NaN,NaN,108.0,104.0,106.0,104.0,100.0,107.0,122.0,110.0,123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111,106,112,NaN,NaN,NaN
320,22267,2024,Tarik Skubal,DET,27,18,4,5.9,2.39,31,31,0,0,0,0,192.0,753,142,54,51,15,35,0,9,2,0,228,218,171,88,22,883,1985,2868,134,13,4,2,10.69,1.64,6.51,6.66,0.70,0.200,0.92,0.273,0.800,2.49,1.27,0.184,0.457,0.358,0.129,0.088,0.060,0.500,52.5,192.0,NaN,NaN,52.5,$46.9,2.48,2.83,4.29,-9.26,13.55,40.04,4.20,0.92,0.92,0.88,0.95,31,4.17,...,102.0,106.0,112.0,107.0,82.0,87.4,12.0,32.0,0.067,113.0,161.0,0.335,481,0.175,0.321,NaN,3.26,66.0,68.0,56.0,NaN,NaN,NaN,60.0,56.0,59.0,52.0,53.0,60.0,54.0,52.0,56.0,60.0,46.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN,61.0,59.0,61.0,-0.55,127.0,100.0,126.0,NaN,NaN,NaN,108.0,105.0,111.0,110.0,101.0,108.0,106.0,101.0,114.0,100.0,103.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,113,102,115,NaN,NaN,NaN
333,10310,2024,Zack Wheeler,PHI,34,16,7,5.4,2.57,32,32,0,0,0,0,200.0,787,139,62,57,20,52,0,8,8,0,224,213,195,92,27,1079,2057,3136,118,17,3,0,10.08,2.34,4.31,6.26,0.90,0.191,0.96,0.246,0.801,3.13,1.09,0.184,0.426,0.390,0.138,0.103,0.080,0.000,49.3,200.0,NaN,NaN,49.3,$43.4,3.08,3.30,3.95,-10.39,14.35,39.13,4.00,0.92,0.90,0.87,0.78,32,3.43,...,115.0,100.0,131.0,95.0,92.0,86.7,13.0,42.0,0.083,114.3,167.0,0.332,503,0.144,0.273,NaN,2.97,NaN,NaN,NaN,48.0,62.0,43.0,67.0,59.0,66.0,64.0,65.0,51.0,64.0,66.0,58.0,NaN,NaN,NaN,45.0,61.0,47.0,52.0,46.0,58.0,64.0,59.0,60.0,-0.74,NaN,NaN,NaN,120.0,92.0,106.0,113.0,108.0,120.0,112.0,103.0,119.0,105.0,110.0,116.0,NaN,NaN,NaN,91.0,98.0,92.0,95.0,102.0,93.0,109,104,113,NaN,NaN,NaN
439,21846,2024,Cole Ragans,KCR,26,11,9,4.9,3.14,32,32,1,0,0,0,186.1,762,146,71,65,15,67,1,6,11,0,223,187,186,86,18,1138,1980,3118,86,10,7,2,10.77,3.24,3.33,7.05,0.72,0.212,1.14,0.290,0.747,2.99,1.01,0.187,0.407,0.405,0.097,0.081,0.053,0.286,44.7,186.1,NaN,NaN,44.7,$39.0,3.16,3.46,2.56,-12.19,14.75,25.74,2.57,1.03,0.98,0.88,1.27,31,2.73,...,92.0,117.0,109.0,99.0,97.0,88.3,15.2,29.0,0.062,115.4,164.0,0.352,466,0.145,0.289,NaN,3.50,61.0,65.0,54.0,NaN,NaN,NaN,61.0,60.0,56.0,NaN,NaN,NaN,53.0,66.0,46.0,52.0,47.0,50.0,51.0,55.0,56.0,NaN,NaN,NaN,58.0,62.0,54.0,-0.38,119.0,95.0,116.0,NaN,NaN,NaN,112.0,97.0,111.0,NaN,NaN,NaN,122.0,105.0,118.0,110.0,100.0,105.0,90.0,102.0,101.0,NaN,NaN,NaN,112,98,111,NaN,NaN,NaN
514,18525,2024,Dylan Cease,SDP,28,14,11,4.8,3.47,33,33,1,1,0,0,189.1,762,137,80,73,18,65,0,2,12,0,224,185,197,83,16,1189,1999,3188,95,16,4,1,10.65,3.09,3.45,6.51,0.86,0.197,1.07,0.2

In [19]:
starters[starters['Name'] == 'Yoshinobu Yamamoto'].head()

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA,botERA,botOvr CH,botStf CH,botCmd CH,botOvr CU,botStf CU,botCmd CU,botOvr FA,botStf FA,botCmd FA,botOvr SI,botStf SI,botCmd SI,botOvr SL,botStf SL,botCmd SL,botOvr KC,botStf KC,botCmd KC,botOvr FC,botStf FC,botCmd FC,botOvr FS,botStf FS,botCmd FS,botOvr,botStf,botCmd,botxRV100,Stf+ CH,Loc+ CH,Pit+ CH,Stf+ CU,Loc+ CU,Pit+ CU,Stf+ FA,Loc+ FA,Pit+ FA,Stf+ SI,Loc+ SI,Pit+ SI,Stf+ SL,Loc+ SL,Pit+ SL,Stf+ KC,Loc+ KC,Pit+ KC,Stf+ FC,Loc+ FC,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
398,33825,2024,Yoshinobu Yamamoto,LAD,25,7,2,2.8,3.0,18,18,0,0,0,0,90.0,368,78,32,30,7,22,0,1,2,0,105,114,75,49,6,493,981,1474,60,7,2,0,10.5,2.2,4.77,7.8,0.7,0.226,1.11,0.305,0.757,2.61,1.52,0.206,0.479,0.315,0.08,0.093,0.061,0.0,25.6,90.0,NaN,NaN,25.6,$22.6,2.84,2.86,0.96,-4.93,5.89,12.16,1.25,0.87,0.82,0.87,0.89,18,1.26,...,102.0,101.0,91.0,101.0,103.0,88.4,8.4,20.0,0.083,112.0,99.0,0.413,240,0.184,0.304,NaN,3.64,NaN,NaN,NaN,64.0,54.0,65.0,55.0,47.0,62.0,68.0,49.0,67.0,49.0,61.0,44.0,NaN,NaN,NaN,52.0,47.0,58.0,58.0,53.0,60.0,57.0,49.0,62.0,-0.29,NaN,NaN,NaN,106.0,104.0,113.0,84.0,109.0,97.0,97.0,119.0,120.0,118.0,96.0,102.0,NaN,NaN,NaN,86.0,111.0,108.0,112.0,101.0,109.0,98,106,105,NaN,NaN,NaN


In [70]:
pitching_stats[pitching_stats['Name'] == 'Yoshinobu Yamamoto'].head()

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA,botERA,botOvr CH,botStf CH,botCmd CH,botOvr CU,botStf CU,botCmd CU,botOvr FA,botStf FA,botCmd FA,botOvr SI,botStf SI,botCmd SI,botOvr SL,botStf SL,botCmd SL,botOvr KC,botStf KC,botCmd KC,botOvr FC,botStf FC,botCmd FC,botOvr FS,botStf FS,botCmd FS,botOvr,botStf,botCmd,botxRV100,Stf+ CH,Loc+ CH,Pit+ CH,Stf+ CU,Loc+ CU,Pit+ CU,Stf+ FA,Loc+ FA,Pit+ FA,Stf+ SI,Loc+ SI,Pit+ SI,Stf+ SL,Loc+ SL,Pit+ SL,Stf+ KC,Loc+ KC,Pit+ KC,Stf+ FC,Loc+ FC,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
398,33825,2024,Yoshinobu Yamamoto,LAD,25,7,2,2.8,3.0,18,18,0,0,0,0,90.0,368,78,32,30,7,22,0,1,2,0,105,114,75,49,6,493,981,1474,60,7,2,0,10.5,2.2,4.77,7.8,0.7,0.226,1.11,0.305,0.757,2.61,1.52,0.206,0.479,0.315,0.08,0.093,0.061,0.0,25.6,90.0,NaN,NaN,25.6,$22.6,2.84,2.86,0.96,-4.93,5.89,12.16,1.25,0.87,0.82,0.87,0.89,18,1.26,...,102.0,101.0,91.0,101.0,103.0,88.4,8.4,20.0,0.083,112.0,99.0,0.413,240,0.184,0.304,NaN,3.64,NaN,NaN,NaN,64.0,54.0,65.0,55.0,47.0,62.0,68.0,49.0,67.0,49.0,61.0,44.0,NaN,NaN,NaN,52.0,47.0,58.0,58.0,53.0,60.0,57.0,49.0,62.0,-0.29,NaN,NaN,NaN,106.0,104.0,113.0,84.0,109.0,97.0,97.0,119.0,120.0,118.0,96.0,102.0,NaN,NaN,NaN,86.0,111.0,108.0,112.0,101.0,109.0,98,106,105,NaN,NaN,NaN
351,33825,2025,Yoshinobu Yamamoto,LAD,26,1,0,0.2,2.7,2,2,0,0,0,0,10.0,41,8,3,3,2,3,0,0,0,0,14,16,5,3,0,58,106,164,3,1,0,0,12.6,2.7,4.67,7.2,1.8,0.211,1.10,0.273,0.976,3.55,3.20,0.125,0.667,0.208,0.00,0.400,0.063,0.0,1.5,10.0,NaN,NaN,1.5,$1.3,3.40,1.79,0.12,-0.56,0.68,1.73,0.19,0.89,0.86,0.87,1.00,2,0.04,...,136.0,36.0,125.0,73.0,130.0,88.2,-4.8,1.0,0.042,111.3,10.0,0.417,24,0.201,0.348,NaN,2.42,NaN,NaN,NaN,61.0,54.0,60.0,57.0,50.0,60.0,52.0,50.0,49.0,80.0,75.0,80.0,NaN,NaN,NaN,74.0,55.0,77.0,59.0,54.0,56.0,60.0,54.0,60.0,-1.00,NaN,NaN,NaN,92.0,111.0,108.0,93.0,117.0,116.0,113.0,102.0,122.0,149.0,200.0,209.0,NaN,NaN,NaN,88.0,144.0,156.0,109.0,108.0,116.0,100,114,118,NaN,NaN,NaN


In [73]:
from pybaseball import statcast_batter

df_batting = statcast_batter('2025-03-27', '2025-04-01', player_id = 660271)

Gathering Player Data


In [74]:
df_batting.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,SL,2025-04-01,79.0,3.12,4.70,"Ohtani, Shohei",660271,519242,single,hit_into_play,NaN,NaN,NaN,NaN,7,Shohei Ohtani singles on a line drive to right...,R,L,L,LAD,ATL,X,9.0,line_drive,0,2,2025,-1.23,-0.50,-0.34,1.84,NaN,NaN,NaN,0,6,Bot,186.56,104.74,NaN,NaN,NaN,NaN,-5.259052,-115.019355,1.496637,-9.930582,21.283336,-37.135605,3.38,1.62,191.0,86.5,12.0,79.3,2369,6.4,778487,686948,621566,645277,663586,669397,650559,671739,672284,54.06,0.666,0.633,0.9,1.0,1.0,0.0,4.0,42,3,Slider,0,1,0,1,1,0,0,1,Infield shade,Standard,307,0.064,0.539,69.8,8.2,0.747,-0.539,88.0,-1,-1,0.387,0.387,36,30,36,31,3,2,NaN,NaN,NaN,NaN,4.15,-1.23,-1.23,NaN
1,CH,2025-04-01,84.3,3.22,4.43,"Ohtani, Shohei",660271,519242,NaN,foul,NaN,NaN,NaN,NaN,6,Shohei Ohtani singles on a line drive to right...,R,L,L,LAD,ATL,S,NaN,NaN,0,1,2025,1.21,-0.33,0.37,2.46,NaN,NaN,NaN,0,6,Bot,NaN,NaN,NaN,NaN,NaN,NaN,-9.052957,-122.518652,2.427580,13.897693,22.349303,-36.165686,3.38,1.62,NaN,NaN,NaN,84.7,1624,6.4,778487,686948,621566,645277,663586,669397,650559,671739,672284,54.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,2,Changeup,0,1,0,1,1,0,0,1,Infield shade,Standard,85,0.000,-0.054,80.6,8.3,NaN,0.054,NaN,-1,-1,0.387,0.387,36,30,36,31,3,2,NaN,NaN,NaN,NaN,3.52,1.21,1.21,NaN
2,SI,2025-04-01,93.1,3.16,4.55,"Ohtani, Shohei",660271,519242,NaN,called_strike,NaN,NaN,NaN,NaN,8,Shohei Ohtani singles on a line drive to right...,R,L,L,LAD,ATL,S,NaN,NaN,0,0,2025,1.29,0.19,0.11,1.60,NaN,NaN,NaN,0,6,Bot,NaN,NaN,NaN,NaN,NaN,NaN,-10.729274,-135.314064,-2.253190,18.090680,26.385185,-29.558914,3.38,1.62,NaN,NaN,NaN,94.0,2100,6.7,778487,686948,621566,645277,663586,669397,650559,671739,672284,53.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,1,Sinker,0,1,0,1,1,0,0,1,Infield shade,Standard,109,0.000,-0.037,NaN,NaN,NaN,0.037,NaN,-1,-1,0.387,0.387,36,30,36,31,3,2,NaN,NaN,NaN,NaN,2.42,1.29,1.29,NaN
3,FF,2025-04-01,95.7,3.08,4.81,"Ohtani, Shohei",660271,519242,field_out,hit_into_play,NaN,NaN,NaN,NaN,6,"Shohei Ohtani grounds out sharply, shortstop N...",R,L,L,LAD,ATL,X,6.0,ground_ball,3,2,2025,1.08,0.60,0.34,2.79,NaN,NaN,NaN,1,3,Bot,121.61,138.66,NaN,NaN,NaN,NaN,-9.676489,-139.076164,-1.090464,16.059662,28.882707,-24.283656,3.38,1.62,55.0,103.8,0.0,96.3,2264,6.5,778487,686948,621566,645277,663586,669397,650559,671739,672284,53.96,0.426,0.387,0.0,1.0,0.0,0.0,4.0,21,8,4-Seam Fastball,0,1,0,1,1,0,0,1,Infield shade,Standard,114,-0.019,-0.332,72.3,7.2,0.471,0.332,103.8,-1,-1,0.403,0.403,36,30,36,31,2,1,NaN,NaN,NaN,NaN,1.88,1.08,1.08,NaN
4,SL,2025-04-01,76.0,3.11,4.71,"Ohtani, Shohei",660271,519242,NaN,foul,NaN,NaN,NaN,NaN

In [75]:
df_batting.to_csv('statcast_batter_shohei.csv')

In [76]:
from pybaseball import team_batting

df_team = team_batting(2024, 2025)

In [77]:
df_team.head()

,teamIDfg,Season,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,...,SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,9,2025,NYY,30,66,135,160,39,17,4,0,18,41,38,19,1,35,3,3,0,1,1,1,0.289,30,55,18,10,671,266,405,3,0,0,0.119,0.219,0.54,0.381,0.719,1.100,0.430,0.247,0.55,0.175,0.291,0.534,0.182,0.327,0.100,0.000,0.452,18.5,37,18.7,-3.1,4.8,-0.4,20.6,2.1,$17.1,3.7,203,1.18,-1.42,2.60,24.78,2.57,0.57,0.09,6,2.04,0.02,0.437,90.9,0.127,...,-0.3,NaN,0.9,NaN,-7.8,8.8,5.1,1.0,NaN,NaN,3.1,-1.8,NaN,0.6,NaN,0.1,4.9,9.3,-0.9,NaN,NaN,1.2,3.7,NaN,0.76,NaN,0.21,2.40,5.59,-12.54,NaN,NaN,1.34,4.51,NaN,0.225,0.676,0.437,0.563,0.850,0.771,0.469,18.3,NaN,124,130,100,123,185,277,92,0.98,69,133,276,121,71,106,123,92,99,91.0,20.7,21,0.204,115.0,50,0.485,103,0.167,0.265,NaN,NaN,NaN,2.1
1,26,2025,PHI,30,61,142,162,41,24,8,1,8,25,23,19,0,35,1,0,0,4,1,1,0.289,45,37,25,4,652,241,411,3,0,0,0.117,0.216,0.54,0.377,0.528,0.905,0.239,0.333,1.22,0.234,0.421,0.346,0.108,0.216,0.067,0.000,0.391,10.6,29,10.3,-0.6,4.9,-0.5,12.3,1.3,$10.2,3.8,154,0.74,-2.99,3.73,6.64,0.69,1.11,0.86,4,1.23,-0.56,0.553,94.9,0.158,...,-0.3,NaN,2.7,NaN,-5.2,8.7,2.4,2.5,NaN,NaN,2.9,0.2,NaN,1.1,NaN,-1.3,5.7,1.9,-1.0,NaN,NaN,1.3,1.3,NaN,1.81,NaN,-1.63,2.18,31.50,-3.34,NaN,NaN,1.28,1.15,NaN,0.244,0.678,0.476,0.635,0.847,0.797,0.535,19.2,NaN,124,123,92,120,134,148,120,1.25,92,97,153,102,95,104,79,103,107,91.3,14.0,14,0.131,112.5,56,0.523,107,0.150,0.247,NaN,NaN,NaN,1.4
2,15,2025,ARI,29,77,171,201,51,28,16,0,7,34,34,22,1,35,3,3,2,2,4,0,0.298,52,56,31,3,759,276,483,3,2,0,0.109,0.174,0.63,0.382,0.515,0.897,0.216,0.333,0.93,0.223,0.374,0.403,0.054,0.125,0.058,0.000,0.384,11.9,35,11.3,1.3,6.1,-0.9,18.0,1.9,$15.0,4.9,148,1.22,-4.19,5.42,12.20,1.24,1.29,2.17,7,1.08,-0.12,0.460,93.4,0.249,...,1.9,NaN,0.6,NaN,-8.0,8.6,6.5,2.0,NaN,NaN,2.5,-1.1,NaN,3.8,NaN,0.3,4.4,1.3,-1.2,NaN,NaN,0.7,2.9,NaN,5.28,NaN,0.44,1.93,1.42,-1.81,NaN,NaN,1.33,1.47,NaN,0.302,0.575,0.445,0.569,0.847,0.757,0.524,18.5,NaN,128,115,74,122,130,133,120,1.19,82,113,89,103,110,82,65,97,124,92.4,14.0,15,0.106,115.7,61,0.433,141,0.187,0.291,NaN,NaN,NaN,1.9
3,2,2025,BAL,30,78,174,195,53,36,7,0,10,32,31,15,0,44,3,3,0,4,3,2,0.305,48,58,27,3,767,274,493,8,0,0,0.077,0.226,0.34,0.364,0.517,0.881,0.213,0.350,0.83,0.203,0.361,0.436,0.052,0.172,0.167,0.000,0.379,10.9,34,11.8,-0.1,5.9,-0.8,18.2,1.9,$15.1,3.6,154,0.57,-2.64,3.21,11.05,1.18,0.78,0.68,4,0.79,-0.05,0.480,94.1,0.181,...,3.1,0.0,-0.2,NaN,-8.6,8.4,4.9,1.6,NaN,NaN,3.1,-0.9,0.0,0.9,NaN,-0.9,3.1,4.2,-1.1,NaN,NaN,0.8,2.3,-0.1,1.52,NaN,-2.66,1.54,5.14,-1.82,NaN,NaN,0.59,1.23,-2.37,0.276,0.644,0.471,0.545,0.839,0.758,0.530,19.2,NaN,131,84,103,118,133,137,130,1.14,86,108,145,94,104,105,78,107,101,89.9,16.1,20,0.150,116.7,53,0.398,133,0.168,0.284,NaN,NaN,NaN,1.9
4,28,2025,STL,28,80,180,205,52,35,9,0,8,30,29,20,2,37,2,2,1,3,5,2,0.289,57,58,28,6,839,309,530,2,3,1,0.098,0.180,0.54,0.363,0.472,0.835,0.183,0.321,0.98,0.196,0.399,0.406,0.103,0.138,0.035,0.333,0.360,8.4,32,8.5,2.8,6.2,-0.5,16.6,1.7,$13.8,4.0,136,0.46,-4.03,4.49,6.62,0.70,1.25,3.87,3,0.88,-0.51,0.483,92.5,0.164,...,1.8,NaN,1.8,NaN,-7.3,7.7,5.4,-1.1,NaN,NaN,1.8,-1.2,NaN,1.1,NaN,0.2,2.0,-1.7,2.

In [79]:
df_team.to_csv('team_batting.csv')